## Goal of investigation

The goal was to compare potential pubsub protocols with respect to utilized erasure coding scheme: 
- Reed-Solomon (RS) erasure coding
- RLNC erasure coding 

The major advantage of RLNC over RS with respect to pubsub is the ability to combine any chunks and get a new validatable chunk. 
Thus way if a node has e.g. 2 (or more) arbitrary chunks then with RS coding the node need to choose which of chunks it wants to propagate, while with RLNC coding the node would just propagate a random linear combination of chunks. The latter should potentially significantly reduce the chance of propagating a duplicate info.        

We also simulate an abstract pubsub protocol without any erasure coding which may be though as similar to the current Ethereum approach of blob dissemination (message chunks ~ DAS columns)

We also consider [Ideal Pubsub](https://hackmd.io/@nashatyrev/rkXUsFD5a) model as a theoretical optimum for a pubsub protocol

We consider very simple network model and very simple pubsub protocol implementations where the only possible message is the raw message chunk. This simplicity should help in reasoning about why this or that statistical pattern is observed and also should help us to outline reasonable directions in developing a real life protocol.           


## TLDR

Utilizing any of two erasure codings for pubsub may yield a significant dissemination performance gain for larger messages. 

Reed-Solomon works great, but RLNC works greater:

      ![compare chart](./images/TldrCompareChart.png)

## Simulation model

- A message is split onto `numberOfChunks` _chunks_ of equal size
- Every node has a _receive FIFO buffer_
- Simulation is executed by _rounds_. 
- Each round every node may enqueue a single chunk to any other node receive buffer.
    - The set of receiving nodes could be limited when simulating _meshes_ 
        - The meshes are randomly generated and are static across all rounds   
- Each round every node pops and processes a single message from its receive buffer if it's not empty
- The above rules simulate nodes' _bandwidth_ equal to 1 chunk/round
- To abstract from the actual bandwidth, message size and chunk count the _time_ is measured as a float value, 
    where `1.0` is the time the full message is transferred from one node to another with zero latency. 
    In other words the time `1.0` corresponds to the period of `numberOfChunks` rounds 
- The _latency_ is simulated in whole rounds (to keep the model simple). When the latency is non-zero the node 
    pops a message from receive buffer only if it was enqueued `latency` rounds ago
- A node has no any information about the state (e.g. buffer size) of other nodes and no any global information besides the current round
- The simulation completes when all the nodes receive the complete message. This is the only 'god' property of the model, when the global network state affects simulation        
       

## Simulation config

- `nodeCount`: total number of nodes in the network
- `numberOfChunks`: number of chunks the message is split onto
- `peerCount` number of nodes in a mesh  
- `erasure` the type of erasure coding and its applicable params: 
    - `NoErasure`: the message chunks are not erasure coded, they are disseminated 'as is' (is treated as a special case of `Rs` with extension factor `1.0`)
        - `meshStrategy`: an attempt to make dynamic mesh size and reduce it in a later dissemination phase 
    - `RS`: Reed-Solomon erasure coding which has a predefined fixed number of extension chunks (in addition to `numberOfChunks` of original message chunks)
        - `extensionFactor`: how many extension chunks are generated. Just `x2` and `x3` were tested
                - `x1` corresponds to `NoErasure`
        - `isDistinctMeshesPerChunk`: defines if distinct messages chunks are propagated via one or distinct meshes
        - `chunkSelectionStrategy`: the strategy of how to select one from existing chunks to propagate at the current round  
    - `RLNC`: RLNC erasure coding     
- `latencyRounds`: the chunk delivery latency measured in rounds
- `peerSelectionStrategy`: how the receiving peer is selected   

## Simulation results

The following key metrics are recorded every round: 
- `doneMsgCnt`: _non duplicate_ message count: the number of messages _received_ which are NOT duplicate for receiving nodes. 
    - when all nodes receive all necessary chunks and may recover original data the count reaches `numberOfChunks * (nodeCount - 1)`
    - this metric is displayed with RED on stacked diagrams
    - `doneMsgFraction`: relative `doneMsgCnt` value `[0.0 .. 1.0]`  
- `dupMsgCnt`: _duplicate_ message count: the number of messages _received_ which don't add any information for receiving nodes and are dropped 
    - `dupBeforeReadyMsgCnt`: (subset of `dupMsgCnt`) is the number of duplicate messages received _before_ a node recovers original message  
    - `dupAfterReadyMsgCnt`: (subset of `dupMsgCnt`) is the number of duplicate messages received _after_ a node recovers original message  

## Simulation

First select the best parameters for each strategy and add some new parameters to optimize performance:  
- [NoErasure](./NoErasure.ipynb)
- [Reed-Solomon](./RS.ipynb)
- [RLNC](./RLNC.ipynb)

Then compare under various angles of view: 
- [Compare](./Compare.ipynb)
 